In [125]:
from SBMLLint.common import constants as cn
from SBMLLint.common.molecule import Molecule, MoleculeStoichiometry
from SBMLLint.common.reaction import Reaction
from SBMLLint.games.som import SOM
from SBMLLint.common.simple_sbml import SimpleSBML

import numpy as np
import pandas as pd
import collections
#
from games_setup import *
from SBMLLint.common.stoichiometry_matrix import StoichiometryMatrix
import SBMLLint.common.constants as cn
import numpy as np
import pandas as pd
import scipy

In [249]:
simple = load_file_from_curated_data(40)
for r in simple.reactions:
  if r.category != cn.REACTION_BOUNDARY:
    print(r.makeIdentifier(is_include_kinetics=False))

Reaction1: Br + BrO3 -> HBrO2 + HOBr
Reaction2: Br + HBrO2 -> 2.00 HOBr
Reaction3: BrO3 + HBrO2 -> Ce + 2.00 HBrO2
Reaction4: 2.00 HBrO2 -> BrO3 + HOBr
Reaction5: Ce -> Br


In [240]:
# x: number of reactants, y: number of products
# r: reactants stoichiometry (list)
# p: products stoichiometry (list)
REACTION_REDUNDANT = "reaction_redundant"
REACTION_ERROR = "reaction_error"
REACTION_SUMMARY_CATEGORIES = [
    cn.ReactionCategory(category=REACTION_REDUNDANT,
        predicate=lambda x,y,r,p: (x==0) and (y==0)),
    cn.ReactionCategory(category=REACTION_ERROR,
        predicate=lambda x,y,r,p: ((x==0) and (y!=0)) \
                                  or ((x!=0) and (y==0))),
    cn.ReactionCategory(category=cn.REACTION_1_1,
        predicate=lambda x,y,r,p: (x==1) and (y==1) and (sum(r)==sum(p))),
    
    cn.ReactionCategory(category=cn.REACTION_1_n,
        predicate=lambda x,y,r,p: (x==1) and (sum([r[0]<=e for e in p])==len(p))  ),
                     
    cn.ReactionCategory(category=cn.REACTION_n_1,
        predicate=lambda x,y,r,p: (y==1) and (sum([p[0]<=e for e in r])==len(r))),
    ]

In [241]:
def getNonBoundaryReactions(simple):
  """
  Get list of non-boundary reacetions
  :param SimpleSBML simple:
  :return list-Reaction:
  """
  reactions = []
  for reaction in simple.reactions:
    if reaction.category != cn.REACTION_BOUNDARY:
      reactions.append(reaction)
  return reactions
#
def getNonBoundaryMolecules(simple, reactions):
  """
  Get list of non-boundary molecules
  :param SimpleSBML simple:
  :return list-Molecule.name:
  """
  molecules = set()
  for reaction in reactions:
    reactants = {r.molecule.name for r in reaction.reactants}
    products = {r.molecule.name for r in reaction.products}
    molecules = molecules.union(reactants)
    molecules = molecules.union(products)
  return list(molecules)
#
def makeStoichiometryMatrix(reactions, molecules):
  """
  Creates a full stoichiometry matrix
  using non-boundary reactions.
  Helped by https://gist.github.com/lukauskas/d1e30bdccc5b801d341d
  :return pd.DataFrame:
  """
  reaction_labels = [r.label for r in reactions]
  stoichiometry_matrix = pd.DataFrame(0.0, index=molecules, columns=reaction_labels)
  for reaction in reactions:
    reactants = {r.molecule.name:r.stoichiometry for r in reaction.reactants}
    products = {p.molecule.name:p.stoichiometry for p in reaction.products}
    reaction_molecules = list(set(reactants.keys()).union(products.keys()))
    for molecule_name in reaction_molecules:
      net_stoichiometry = products.get(molecule_name, 0.0) - reactants.get(molecule_name, 0.0)
      stoichiometry_matrix[reaction.label][molecule_name] = net_stoichiometry
  return stoichiometry_matrix
def decomposeMatrix(mat_df):
  """
  LU decomposition of the matrix.
  First it transposes the input matrix
  and find P, L, U matrices. 
  :param pandas.DataFrame mat_df:
  :yield typle-numpy.array:
  """
  mat_t = mat_df.T
  idx_mat_t = mat_t.index
  # LU decomposition
  mat_lu = scipy.linalg.lu(mat_t)
  # inverse pivot matrix
  p_inv = scipy.linalg.inv(mat_lu[0])
  pivot_index = [list(k).index(1) for k in p_inv]
  new_idx_mat_t = [idx_mat_t[idx] for idx in pivot_index]
  # row reduced matrix
  row_reduced = pd.DataFrame(mat_lu[2], index=new_idx_mat_t, columns=mat_t.columns).T
  # 'L' matrix
  yield mat_lu[1]
  yield row_reduced
#
# we need a different kind of category for this
def getReactionSummaryCategory(reactants, products):
  """
  Return category of reaction. Return reaction_n_n
  if none of the above applies
  :param list-MoleculeStoichiometry reactants:
  :param list-Moleculestoichiometry products:
  :return str reaction_category:
  """
  num_reactants = len([r.molecule for r in reactants \
                       if r.molecule.name!=cn.EMPTYSET])
  num_products = len([p.molecule for p in products \
                      if p.molecule.name!=cn.EMPTYSET])
  stoichiometry_reactants = [r.stoichiometry for r \
                                in reactants \
                                if r.molecule.name!=cn.EMPTYSET]
  stoichiometry_products = [p.stoichiometry for p \
                           in products \
                           if p.molecule.name!=cn.EMPTYSET]
  for reaction_category in REACTION_SUMMARY_CATEGORIES:
    if reaction_category.predicate(num_reactants, num_products, 
                                   stoichiometry_reactants, 
                                   stoichiometry_products):
      return reaction_category.category
  return cn.REACTION_n_n
#    
ReactionSummary = collections.namedtuple('ReactionSummary', 
                  'label reactants products category')
def convertMatrixToReactions(simple, mat_df):
  """
  Convert a stoichiometry matrix, 
  where columns are reactions and 
  rows are molecules(species),
  to simpleSBML reactions. 
  :param simpleSBML simple:
  :param pandas.DataFrame mat_df:
  :return list-ReactionSummary reactions:
  """
  reactions = []
  for reaction_name in mat_df.columns:
    reaction = simple.getReaction(reaction_name)
    reduced_reaction_series = mat_df[reaction_name]
    reactants = [MoleculeStoichiometry(simple.getMolecule(molecule), 
                                   abs(reduced_reaction_series[molecule])) \
            for molecule in reduced_reaction_series.index if reduced_reaction_series[molecule]<0]
    products = [MoleculeStoichiometry(simple.getMolecule(molecule), 
                                   reduced_reaction_series[molecule]) \
            for molecule in reduced_reaction_series.index if reduced_reaction_series[molecule]>0]
    reactions.append(ReactionSummary(label=reaction_name, 
                                    reactants=reactants,
                                    products=products,
                                    category=getReactionSummaryCategory(reactants, products)))
  return reactions

In [244]:
reactions = getNonBoundaryReactions(simple)
molecules = getNonBoundaryMolecules(simple, reactions)
matrix = makeStoichiometryMatrix(reactions, molecules)
lower, reduced = decomposeMatrix(matrix)
print(lower)
print(reduced.T)
reduced_reactions = convertMatrixToReactions(simple, reduced)
#reduced_reactions
for r in reduced_reactions:
  if r.category == REACTION_ERROR:
    print("\n\nError found!!")
    print(r.reactants)
    print(r.products)
    print(r.category)
print("\n***printed errors above***")

[[ 1.    0.    0.    0.  ]
 [-0.5   1.    0.    0.  ]
 [ 0.   -0.    1.    0.  ]
 [ 0.75  0.5  -0.    1.  ]]
             PGA  CO2  NADP  NADPH  RuBP
PGA_prod_Vc  2.0 -1.0   0.0   -2.0  -1.0
PGA_cons     0.0 -0.5   0.0   -1.0   0.5
NADPH_prod   0.0  0.0  -1.0    1.0   0.0
PGA_prod_Vo  0.0  0.0   0.0    0.0  -0.5


Error found!!
[RuBP *  0.50]
[]
reaction_error

***printed errors above***


In [248]:
# Now created SOMGraph using reduced reactions
def initializeSOMs(molecules):
  """
  Create a list of one-molecule SOMs
  :param SimpleSBML simple:
  :return list-SOM:
  """
  soms = []
  for molecule in simple.molecules:
    if molecule.name == cn.EMPTYSET:
      continue
    else:
      soms.append(SOM({molecule}))
  return soms
#
def processReactions(reactions):
  """
  Process reactions, find error
  :param list-ReactionSummary reactions:
  :return ??:
  """
  uniuni = []
  unimulti = []
  multiuni = []
  for reaction in reactions:
    if reaction.category == cn.REACTION_1_1:
      uniuni.append(reaction)
    elif reaction.category == cn.REACTION_1_n:
      unimulti.append(reaction)
    elif reaction.category == cn.REACTION_n_1:
      multiuni.append(reaction)
  # next, process these using "process reaction methods"
  return

In [234]:
soms = initializeSOMs(molecules)
soms

[{Br}, {BrO3}, {HBrO2}, {HOBr}, {Ce}]

In [122]:
matrix.T

,HBrO2,Ce,HOBr,Br,BrO3
Reaction1,1.0,0.0,1.0,-1.0,-1.0
Reaction2,-1.0,0.0,2.0,-1.0,0.0
Reaction3,1.0,1.0,0.0,0.0,-1.0
Reaction4,-2.0,0.0,1.0,0.0,1.0
Reaction5,0.0,-1.0,0.0,1.0,0.0


In [44]:
mat_t = matrix.T
idx_mat_t = mat_t.index
# LU decomposition
mat_lu = scipy.linalg.lu(mat_t)
print(idx_mat_t)
print(type(idx_mat_t))
type(mat_lu)

Index(['Reaction1', 'Reaction2', 'Reaction3', 'Reaction4', 'Reaction5'], dtype='object')
<class 'pandas.core.indexes.base.Index'>


tuple

In [45]:
mat_lu[0]

array([[0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0.]])

In [46]:
mat_lu[1]

array([[ 1.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [-0.5       ,  1.        ,  0.        ,  0.        ,  0.        ],
       [ 0.5       ,  0.        ,  1.        ,  0.        ,  0.        ],
       [-0.        , -1.        ,  0.33333333,  1.        ,  0.        ],
       [-0.5       ,  0.        ,  1.        ,  0.        ,  1.        ]])

In [47]:
mat_lu[2]

array([[-2.        ,  0.        ,  1.        ,  0.        ,  1.        ],
       [ 0.        ,  1.        ,  0.5       ,  0.        , -0.5       ],
       [ 0.        ,  0.        ,  1.5       , -1.        , -0.5       ],
       [ 0.        ,  0.        ,  0.        ,  1.33333333, -0.33333333],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ]])

In [49]:
mat_t

,HBrO2,Ce,HOBr,Br,BrO3
Reaction1,1.0,0.0,1.0,-1.0,-1.0
Reaction2,-1.0,0.0,2.0,-1.0,0.0
Reaction3,1.0,1.0,0.0,0.0,-1.0
Reaction4,-2.0,0.0,1.0,0.0,1.0
Reaction5,0.0,-1.0,0.0,1.0,0.0


In [97]:
abs(-1)

1

In [104]:
NAME = 'Reaction1'
reaction = simple.getReaction(NAME)
print("current reaction is")
print(reaction)
reduced_reaction_series = reduced_mat.T[NAME]
print(reduced_reaction_series)
reactants = [MoleculeStoichiometry(simple.getMolecule(molecule), 
                                   abs(reduced_reaction_series[molecule])) \
            for molecule in reduced_reaction_series.index if reduced_reaction_series[molecule]<0]
products = [MoleculeStoichiometry(simple.getMolecule(molecule), 
                                   reduced_reaction_series[molecule]) \
            for molecule in reduced_reaction_series.index if reduced_reaction_series[molecule]>0]
# if !reactants or !products then error message or quit
# if not, do the followings
reaction.reactants = reactants
reaction.products = products
reaction.identifier = reaction.makeIdentifier(is_include_kinetics=True)
print("reduced reaction is")
print(reaction)

current reaction is
Reaction1:  -> ; Br * BrO3 * k1 * BZ
HBrO2    0.0
Ce       0.0
HOBr     0.0
Br       0.0
BrO3     0.0
Name: Reaction1, dtype: float64
reduced reaction is
Reaction1:  -> ; Br * BrO3 * k1 * BZ


In [75]:
mat_converted = convertMatrixToReactions(mat_t)
for row_index in mat_converted.columns:
  pass
  reaction = mat_converted[row_index]
  #print(reaction)
  #print(reaction[0])
  #print(type(mat_converted[row_index]))
  #print(mat_converted[row_index])
mat_converted

,Reaction1,Reaction2,Reaction3,Reaction4,Reaction5
HBrO2,1.0,-1.0,1.0,-2.0,0.0
Ce,0.0,0.0,1.0,0.0,-1.0
HOBr,1.0,2.0,0.0,1.0,0.0
Br,-1.0,-1.0,0.0,0.0,1.0
BrO3,-1.0,0.0,-1.0,1.0,0.0


In [100]:
p_inv = scipy.linalg.inv(mat_lu[0])
pivot_index = [list(k).index(1) for k in p_inv]
new_idx_mat_t = [idx_mat_t[idx] for idx in pivot_index]
#
reduced_mat = pd.DataFrame(mat_lu[2], index=new_idx_mat_t, columns=mat_t.columns)
reduced_mat

,HBrO2,Ce,HOBr,Br,BrO3
Reaction4,-2.0,0.0,1.0,0.000000,1.000000
Reaction3,0.0,1.0,0.5,0.000000,-0.500000
Reaction2,0.0,0.0,1.5,-1.000000,-0.500000
Reaction5,0.0,0.0,0.0,1.333333,-0.333333
Reaction1,0.0,0.0,0.0,0.000000,0.000000
